In [1]:
import pandas as pd
from COMETH import Skeleton,DynamicSkeleton
import nimblephysics as nimble
import numpy as np
import json
import time
import csv
gui = nimble.NimbleGUI()
gui.serve(8080)

# sudo lsof -iTCP:8080 -sTCP:LISTEN

GUIWebsocketServer will start serving a WebSocket server on ws://localhost:8070
Web GUI serving on http://localhost:8080


In [ ]:
gui.nativeAPI().clear()
# Build skeleton
s12 = Skeleton('BODY12.xml')
# BSM
s = DynamicSkeleton(config='BODY15_constrained_3D.xml')
s.hip_correction = False
# gui.nativeAPI().renderSkeleton(s._nimble)
axis_length=0.2
gui.nativeAPI().createLine(key='x_axis',points=[np.array([0,0,0]),np.array([axis_length,0,0])],color=np.array([255,0,0,1]))
gui.nativeAPI().createLine(key='y_axis',points=[np.array([0,0,0]),np.array([0,axis_length,0])],color=np.array([0,255,0,1]))
gui.nativeAPI().createLine(key='z_axis',points=[np.array([0,0,0]),np.array([0,0,axis_length])],color=np.array([0,0,255,1]))

# Read data from CSV
# markers = pd.read_csv("~/nas/MAEVE/BEFINE/wearable_camera/TotalCapture/CommonFormat/vicon/s1/acting1/vicon_s1_acting1.csv")
# imus = pd.read_csv("~/nas/MAEVE/BEFINE/wearable_camera/TotalCapture/CommonFormat/imu/s1/acting1/imu_s1_acting1.csv")
markers = pd.read_csv("tmp/imu/vicon_s1_acting1.csv")
imus = pd.read_csv("tmp/imu/imu_s1_acting1.csv")

Setting len to 0.1
Setting neutral pos to   0
0.7
  0
Setting len to 0.3
Setting neutral pos to    0
-0.6
   0
Setting len to 0.1
Setting neutral pos to   0
0.2
  0


127.0.0.1 - - [23/Jul/2025 09:05:43] "GET /?vscodeBrowserReqId=1753254344422 HTTP/1.1" 200 -


In [3]:
# Build the markers dataframe with only the subset we are interested in

Rx = np.array([
    [1,0,0 ],
    [0,0,-1 ],
    [0,1,0]
])

markers.columns
markers_dict = {
'RKnee': 'right_knee',
'LWrist': 'left_wrist',
'RHip': 'right_hip',
'RShoulder': 'right_shoulder',
'LElbow': 'left_elbow',
'LHip': 'left_hip',
'RElbow': 'right_elbow',
'RWrist': 'right_wrist',
'LKnee': 'left_knee',
'LShoulder': 'left_shoulder',
'RAnkle': 'right_ankle',
'LAnkle': 'left_ankle'
 }
target = []
for i in range(markers.shape[0]):
    row = []
    for kp in markers_dict.keys():
        p = np.array([markers[markers_dict[kp]+"_x"][i],markers[markers_dict[kp]+"_y"][i],markers[markers_dict[kp]+"_z"][i]])
        # rotate the 3d point -90 on the x axis (from y up to z up)
        p_n = Rx.dot(p)
        # print(p,p_n)
        row += p_n.tolist()
    target.append(row)
target = np.array(target)

In [4]:
# Place virtual IMUs on the shoulders
from typing import List, Tuple
right_shoulder: nimble.dynamics.BodyNode = s._nimble.getBodyNode("humerus_r")
translation: np.ndarray = np.array([0.0, 0.0, 0.025])
# z is pointing up, x lateral and y frontal
rotation: np.ndarray = np.array([[1,0,0],[0,0,1],[0,-1,0]]) @ np.array([[0,1,0],[-1,0,0],[0,0,1]])
watch_offset: nimble.math.Isometry3 = nimble.math.Isometry3(rotation, translation)
print(watch_offset)
sensors: List[Tuple[nimble.dynamics.BodyNode, nimble.math.Isometry3]] = [(right_shoulder, watch_offset),]

[[0.    1.    0.    0.   ]
 [0.    0.    1.    0.   ]
 [1.    0.    0.    0.025]
 [0.    0.    0.    1.   ]]


127.0.0.1 - - [23/Jul/2025 09:05:55] "GET /?vscodeBrowserReqId=1753254356300 HTTP/1.1" 200 -


In [5]:
# Plot imu position with axes
def plot_imu(imu,axis_length=0.1,imu_key='imu'):
    j_w = imu[0].getWorldTransform().matrix()   # From joint to world
    imu_j = imu[1].matrix()                     # From IMU to joint
    imu_w = j_w @ imu_j # From IMU to world
    
    # Origin
    p0 = imu_w @ np.eye(4)
    gui.nativeAPI().createSphere(key=imu_key+'o', radii=np.array([0.005,0.005,0.005]),pos=p0[0:3,3],color=np.array([0,0,0,1]))
    
    # X coordinate
    px = np.eye(4)
    px[0:3,3] = [axis_length,0,0]
    px = imu_w @ px
    gui.nativeAPI().createLine(key=imu_key+'x',points=[p0[0:3,3],px[0:3,3]],color=np.array([255,0,0,1]))
    
    # Y coordinate
    py = np.eye(4)
    py[0:3,3] = [0,axis_length,0]
    py = imu_w @ py
    gui.nativeAPI().createLine(key=imu_key+'y',points=[p0[0:3,3],py[0:3,3]],color=np.array([0,255,0,1]))
    
    # Z coordinate
    pz = np.eye(4)
    pz[0:3,3] = [0,0,axis_length]
    pz = imu_w @ pz
    gui.nativeAPI().createLine(key=imu_key+'z',points=[p0[0:3,3],pz[0:3,3]],color=np.array([0,0,255,1]))
    
# plot_imu(sensors[0],axis_length=0.05,imu_key='imu'+str(0))

In [6]:
pos_imu = right_shoulder.getWorldTransform().multiply(watch_offset).matrix()[0:3,3]
s._nimble.setGravity(np.array([0.0, 0.0, -9.81]))
s._nimble.getAccelerometerReadings(sensors)

array([-1.94980196,  4.50886163, -8.5018066 ])

In [7]:
# Move the body model using gt to the first position
kps = list(markers_dict.keys())
s.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale()

127.0.0.1 - - [23/Jul/2025 09:06:06] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:06:06] "GET /v1/health HTTP/1.1" 404 -


In [8]:
# giving the absolute position and orientation of the IMU, get the angle between the floor and the imu Z axis
def get_alpha_angle(imu, plot=True):
    j_w = imu[0].getWorldTransform().matrix()   # From joint to world
    imu_j = imu[1].matrix()                     # From IMU to joint
    imu_w = j_w @ imu_j # From IMU to world
    
    origin = imu_w[:3,3]
    origi_proj = origin.copy()
    origi_proj[-1] = 0 # z = 0
        
    pz = np.eye(4)
    pz[0:3,3] = [0,0,-0.3]
    pz = imu_w @ pz
    
    if plot:
        gui.nativeAPI().createLine(key='origin_projected',points=[origin,origi_proj],color=np.array([0,0,0,0.8]))
        gui.nativeAPI().createLine(key='origin_projected2',points=[origin,pz[:3,3]],color=np.array([0,0,0,0.8]))
    
    # Vectors AB and BC
    u = origi_proj - origin
    v = pz[:3,3] - origin

    # Compute angle in radians
    cos_theta = np.dot(u, v) / (np.linalg.norm(u) * np.linalg.norm(v))
    theta_rad = np.arccos(np.clip(cos_theta, -1.0, 1.0))  # Clip to avoid numerical errors

    # Optional: convert to degrees
    theta_deg = np.degrees(theta_rad)
    return theta_deg
    

In [ ]:
def estimate_alpha_angle(a):
    alpha = np.arctan2(np.sqrt(a[0]**2+a[1]**2),a[2])
    return np.degrees(alpha)
# estimate_angle(s._nimble.getAccelerometerReadings(sensors))

127.0.0.1 - - [23/Jul/2025 09:06:36] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:06:36] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2025 09:07:06] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:07:06] "GET /v1/health HTTP/1.1" 404 -


In [ ]:
# Get ALPHA

# Move right shoulder up
q = s._nimble.getPositions()
q[32] = 0*np.pi/180 # adduction/abduction
q[34] = 0*np.pi/180
q[33] = 20*np.pi/180
s._nimble.setPositions(q)

acc_0 = s._nimble.getAccelerometerReadings(sensors)

# Rendering scene
for i in range(0,-90,-1):
    q[32] = i*np.pi/180
    s._nimble.setPositions(q)
    gui.nativeAPI().renderSkeleton(s._nimble)
    plot_imu(sensors[0],axis_length=0.05,imu_key='imu'+str(0))
    gt_angle = get_alpha_angle(sensors[0],plot=False)
    print(estimate_alpha_angle(s._nimble.getAccelerometerReadings(sensors)),gt_angle)
    time.sleep(0.1)
    print(s._nimble.getAccelerometerReadings(sensors))

26.12894102490144 26.212144614836962
[3.82450823 2.00277284 8.80118636]
26.813267743417903 26.89825471350665
[3.9684271  1.95039066 8.74866896]
27.515602089081007 27.602149105050017
[4.11146278 1.89832993 8.69348663]
28.234597455589647 28.32249536405725
[4.25357168 1.84660652 8.63565618]
28.969010169114412 29.058063318816803
[4.39471053 1.79523618 8.57519523]
29.717693762653848 29.807719246989382
[4.53483633 1.74423456 8.51212219]
30.479592858902198 30.570419718816403
[4.67390639 1.69361719 8.44645627]
31.25373690821752 31.345205324732863
[4.81187836 1.6433995  8.37821748]
32.03923396361231 32.13119446135257
[4.94871021 1.59359678 8.30742661]
32.83526462241136 32.927577299111114
[5.08436025 1.54422421 8.23410521]
33.64107622291419 33.733610014929
[5.21878717 1.49529681 8.15827563]
34.45597735244347 34.548609342461496
[5.35195002 1.4468295  8.07996095]
35.279332698948096 35.371947469237746
[5.48380824 1.39883703 7.99918504]
36.110558260380365 36.20304729280324
[5.61432165 1.35133403 7.9

127.0.0.1 - - [23/Jul/2025 09:07:36] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:07:36] "GET /v1/health HTTP/1.1" 404 -


[8.72465189 0.21926641 4.4369078 ]
63.98530116925647 64.04140959786645
[8.79625072 0.19320656 4.29404736]
64.91997143304565 64.9742601276515
[8.86549576 0.16800343 4.14987892]
65.85571580281828 65.90816031164636
[8.9323659  0.14366469 4.00444638]
66.79247405555985 66.84305097843023
[8.99684078 0.12019775 3.85779405]
67.73018866846812 67.77887563677876
[9.05890076 0.09760977 3.7099666 ]
68.66880462131523 68.71558027989494
[9.11852693 0.07590762 3.56100905]
69.60826921272127 69.65311320338236
[9.17570112 0.05509791 3.41096679]
70.54853188909807 70.5914248357354
[9.23040594 0.03518699 3.25988552]
71.48954408513312 71.53046758023088
[9.2826247  0.01618091 3.10781125]
72.43125907478306 72.47019566720431
[ 9.33234151e+00 -1.91452696e-03  2.95479031e+00]
73.37363183183706 73.41056501578225
[ 9.37954122 -0.01909382  2.80086932]
74.31661889918917 74.35153310422129
[ 9.42420945 -0.03535172  2.64609515]
75.26017826603295 75.29305884807695
[ 9.4663326  -0.05068329  2.49051496]
76.20426925225459 76

127.0.0.1 - - [23/Jul/2025 09:08:06] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:08:06] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2025 09:08:36] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:08:36] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2025 09:09:06] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:09:06] "GET /v1/health HTTP/1.1" 404 -
127.0.0.1 - - [23/Jul/2025 09:09:36] code 404, message File not found
127.0.0.1 - - [23/Jul/2025 09:09:36] "GET /v1/health HTTP/1.1" 404 -


In [ ]:
# Move right shoulder down
s.reset()
s12.load_from_numpy(target[0,:].reshape(-1,3),s.kps)
s.load_from_BODY12(s12)
s.exact_scale()
q = s._nimble.getPositions()
q[32] = 0*np.pi/180 # adduction/abduction
q[34] = 0*np.pi/180
q[33] = 20*np.pi/180
s._nimble.setPositions(q)

acc_0 = s._nimble.getAccelerometerReadings(sensors)

q[32] += 15*np.pi/180
s._nimble.setPositions(q)

for i in range(1000):
    gui.nativeAPI().renderSkeleton(s._nimble)
    q = s._nimble.getPositions()
    acc_target = s._nimble.getAccelerometerReadings(sensors)

    error = acc_0 - acc_target
    loss = np.inner(error, error)

    d_loss_d_acc = 2 * (acc_0 - acc_target)
    d_acc_d_joint_angles = s._nimble.getAccelerometerReadingsJacobianWrt(accs=sensors,wrt=nimble.neural.WRT_POSITION)
    d_loss_d_joint_angles = d_acc_d_joint_angles.T @ d_loss_d_acc

    d_loss_d_joint_angles[0:6] = 0.0 # Do not update the pelvis position in the worlds

    q += 0.00001 * d_loss_d_joint_angles
    s._nimble.setPositions(q)
    # time.sleep(0.1)
    print("loss:",loss,)